# Filling missing information from the tokens

## Libraries

In [48]:
import pandas as pd
import numpy as np
import os
import json
import requests

## Loading all the chains and its token

In [49]:
# Function to load JSON
def loadJSON(filepath):
    with open(filepath) as file:
        return json.load(file)

In [50]:
df = pd.DataFrame(loadJSON('../../data/json/tvl-chains-symbol.json'))

print(df.shape)
df.head()

(245, 6)


,gecko_id,tvl,tokenSymbol,cmcId,name,chainId
0,harmony,1.895183e+06,ONE,3945,Harmony,1666600000
1,mantle,4.613355e+08,MNT,27075,Mantle,5000
2,aurora-near,1.671190e+07,AURORA,14803,Aurora,1313161554
3,moonbeam,2.557276e+07,GLMR,6836,Moonbeam,1284
4,kava,1.488895e+08,KAVA,4846,Kava,None


## Getting the rows without cmcId

In [51]:
null_cmcId_df = df[df['cmcId'].isnull()]
null_cmcId_df

,gecko_id,tvl,tokenSymbol,cmcId,name,chainId
45,None,1.346509e+08,RON,None,Ronin,2020
46,evmos,7.567913e+05,EVMOS,None,Evmos,9001
56,zencash,2.453932e+06,ZEN,None,Horizen EON,7332
71,None,5.951530e+05,QGOV,None,Q Protocol,35441
75,injective-protocol,4.237226e+07,INJ,None,Injective,None
99,dogechain,2.533325e+06,DG,None,Dogechain,None
102,None,0.000000e+00,EMPIRE,None,Empire,None
131,zklink,4.151204e+08,ZKL,None,zkLink Nova,810180
133,mode,4.296166e+08,MODE,None,Mode,34443
144,g-token,1.118389e+06,G,None,Gravity,1625


## Preparing the tokens to be passed to the endpoint

In [52]:
# Setting up the API key from the env
API_KEY = os.getenv('COINMARKETCAP_MARCO_KEY')

In [53]:
# Getting all the symbols
symbols = null_cmcId_df['tokenSymbol'].tolist()

# Convert the list of symbols to a comma-separated string to pass it to the endpoint
symbol_string = ','.join(symbols)

# Display the string of symbols
print("Comma-separated string of symbols:")
print(symbol_string)

Comma-separated string of symbols:
RON,EVMOS,ZEN,QGOV,INJ,DG,EMPIRE,ZKL,MODE,G,KRO,DXT,PLS,HKO,PARA,CRE,STRK,DEL,ALV,TLC,ZENIQ,LIBRE,BONE,WHALE,MC,NLS,CACAO,DEGEN,JBC,BTN,MANTA,ROSE,RSS3,LAC,RWA,IDEX


## Requesting token information

In [54]:
# URL and headers for the CoinMarketCap API call
url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/info'

# API key from environment
API_KEY = os.getenv('COINMARKETCAP_MARCO_KEY')


headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': API_KEY,
}

# Parameters for the request
parameters = {
    'symbol': symbol_string,
}

In [55]:
# Make the request
response = requests.get(url, headers=headers, params=parameters)

In [56]:
# Check the status of the response
api_response = {}
if response.status_code == 200:
    result = response.json()
    
    # Create a dictionary with the API response and the counter
    api_response = {
        'data': result.get('data', {}),
    }

    # Process and count the information
    for symbol, info in result['data'].items():
        api_response['data'][symbol] = info
    
    print(f"Successful")
else:
    print(f"Error: {response.status_code} - {response.json()}")

Successful


## Fill CoinMarketCap ID
As these tokens do not have a specific cmcId, we are going to get several tokens with the same symbol. In order to get a single one, we look for a coincidence from the name from Defillama and the one from CoinMarketCap.

In [57]:
# Storing tokens without coincidence
symbol_without_coincidence = {'data': {}}

for symbol, coins in api_response['data'].items():
    # Identifying the row with the current symbol and its id
    row_with_symbol = null_cmcId_df[null_cmcId_df['tokenSymbol'] == symbol]
    row_with_symbol_id = row_with_symbol.index[0]

    for i, coin in enumerate(coins):
        # Check if the the CoinMarketCap token name includes the one from Defillama
        if row_with_symbol['name'].str.lower().iloc[0] in coin['name'].lower():
            # Replacing the cmcId on the original df and the null one
            df.at[row_with_symbol_id, 'cmcId'] = coin['id']
            null_cmcId_df.at[row_with_symbol_id, 'cmcId'] = coin['id']
            break
        # Add to the dictionary in case there's no coincidence
        if i == len(coins)-1: symbol_without_coincidence['data'][symbol] = coins

In [58]:
# Checking what has changed
null_cmcId_df

,gecko_id,tvl,tokenSymbol,cmcId,name,chainId
45,None,1.346509e+08,RON,14101,Ronin,2020
46,evmos,7.567913e+05,EVMOS,19899,Evmos,9001
56,zencash,2.453932e+06,ZEN,None,Horizen EON,7332
71,None,5.951530e+05,QGOV,29443,Q Protocol,35441
75,injective-protocol,4.237226e+07,INJ,7226,Injective,None
99,dogechain,2.533325e+06,DG,None,Dogechain,None
102,None,0.000000e+00,EMPIRE,10613,Empire,None
131,zklink,4.151204e+08,ZKL,None,zkLink Nova,810180
133,mode,4.296166e+08,MODE,31016,Mode,34443
144,g-token,1.118389e+06,G,32120,Gravity,1625


Now there's less null values in **cmcId!!** :)

In [59]:
# Getting again only the null values
null_cmcId_df = df[df['cmcId'].isnull()]
null_cmcId_df

,gecko_id,tvl,tokenSymbol,cmcId,name,chainId
56,zencash,2.453932e+06,ZEN,None,Horizen EON,7332
99,dogechain,2.533325e+06,DG,None,Dogechain,None
131,zklink,4.151204e+08,ZKL,None,zkLink Nova,810180
164,crescent-network,5.557977e+04,CRE,None,Crescent,None
178,decimal,1.314185e+00,DEL,None,DSC,None
199,white-whale,3.913340e+05,WHALE,None,Migaloo,None
206,None,3.156900e+06,MC,None,Beam,4337
211,cacao,1.961941e+07,CACAO,None,Mayachain,None
217,None,2.249053e+04,JBC,None,JBC,None
221,bitnet,4.171326e+03,BTN,None,Bitnet,None


## Checking and replacing manually
Comparing information from Defillama and CoinMarket to see if there's a coincidence and they have market cap information.

### Zen - Horizen

In [60]:
df.at[56, 'cmcId'] = '1698'

### DG - Dogechain

In [61]:
df.at[99, 'cmcId'] = '21414'
df.at[99, 'tokenSymbol'] = "DC"

### WHALE - White Whale

In [62]:
df.at[199, 'cmcId'] = '16121'

### BEAM - Beam

In [63]:
df.at[206, 'cmcId'] = '28298'
df.at[206, 'tokenSymbol'] = "BEAM"

The rest doesn't have enough information or are different from the ones in CoinMarketCap and CoinGecko

In [64]:
# Dropping the rows with null walues in cmcId since we cannot replace them
df = df.dropna(subset=['cmcId'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 235 entries, 0 to 244
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   gecko_id     227 non-null    object 
 1   tvl          235 non-null    float64
 2   tokenSymbol  235 non-null    object 
 3   cmcId        235 non-null    object 
 4   name         235 non-null    object 
 5   chainId      97 non-null     object 
dtypes: float64(1), object(5)
memory usage: 12.9+ KB


We finally have all the cmcIds complete!

In [65]:
# Converting all to strings
df['cmcId'] = df['cmcId'].apply(lambda x: str(x))

## Checking duplicated values

In [66]:
duplicates = df[df.duplicated(subset=['cmcId'], keep=False)]
duplicates.sort_values(by='cmcId')

,gecko_id,tvl,tokenSymbol,cmcId,name,chainId
68,bitcoin-cash,1.220516e+07,BCH,1831,Bitcoincash,None
101,bitcoin-cash,4.555532e+06,BCH,1831,smartBCH,10000
5,binancecoin,4.596636e+09,BNB,1839,BSC,56
33,binancecoin,0.000000e+00,BNB,1839,Binance,56
51,flare-networks,1.670018e+07,FLR,4172,Flare,None
55,terra-luna,2.339099e+06,LUNC,4172,Terra Classic,None


In [67]:
# There's information for Bitcoincash but not for smartBCH in CoinMarketCap
df.drop(index=101, inplace=True)

In [68]:
# The information of BSC is not in CoinMarketCap, but the market cap of Binance is more than 80 000 000 000, so we can get rid of it
df.drop(index=5, inplace=True)
df.drop(index=33, inplace=True)

In [69]:
# Incorrect id
df.at[51, 'cmcId'] = '7950'

## Save to JSON

In [70]:
df.to_json('../../data/json/main-defi-tokens.json', orient='records', indent=4)